"웹 크롤러"
뉴스 데이터 크롤링 및 경제/금융 문제 & 해설 데이터셋 수집 by. 권수현 / 차시현

In [5]:
!pip install beautifulsoup4 fake-useragent pandas openpyxl lxml XlsxWriter

In [59]:
from bs4 import BeautifulSoup
import requests
import openpyxl
from openpyxl import Workbook
import pandas as pd
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows

In [66]:
def generate_naver_news_url(query, start_date, end_date, sort=0):
    base_url = "https://search.naver.com/search.naver"
    query_param = f"query={query.replace(' ', '+')}"
    ds_param = f"ds={start_date}"
    de_param = f"de={end_date}"
    sort_param = f"sort={sort}"

    url = f"{base_url}?where=news&{query_param}&{ds_param}&{de_param}&{sort_param}&start=1"
    return url

In [68]:
def crawl_news(query, start_date, end_date, page_limit=21):
    news_data = []

    for page in range(page_limit):
        start = page * 10 + 1
        search_url = generate_naver_news_url(query, start_date, end_date)[:-1] + str(start)

        print(f"크롤링 중: {search_url}")
        resp = requests.get(search_url)
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        totalNews = soup.select('div.news_area')

        for m in totalNews:
            title = m.select_one('a.news_tit').text
            url = m.select_one('a.news_tit')['href']
            message = m.select_one('div.dsc_wrap > a.api_txt_lines.dsc_txt_wrap').text
            press = m.select_one('a.info.press').text

            news_data.append({
                'title': title,
                'url': url,
                'message': message,
                'press': press
            })

    return news_data

In [70]:
if __name__ == "__main__":
    query = input("검색할 쿼리를 입력하세요: ")
    start_date = input("시작 날짜를 입력하세요 (YYYY.MM.DD 형식): ")
    end_date = input("종료 날짜를 입력하세요 (YYYY.MM.DD 형식): ")
    
    news_results = crawl_news(query, start_date, end_date)
    
    # 결과를 DataFrame으로 변환하고 Excel로 저장
    df = pd.DataFrame(news_results)
    df.to_excel("news_results.xlsx", index=False)
    print("크롤링 완료 및 저장: news_results.xlsx")

검색할 쿼리를 입력하세요:  경제
시작 날짜를 입력하세요 (YYYY.MM.DD 형식):  2012.01.01
종료 날짜를 입력하세요 (YYYY.MM.DD 형식):  2024.10.39


크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=1
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=11
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=21
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=31
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=41
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=51
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=61
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=71
크롤링 중: https://search.naver.com/search.naver?where=news&query=경제&ds=2012.01.01&de=2024.10.39&sort=0&start=81
크롤링 중: https://searc

In [ ]:
import pandas as pd
import json

def load_news_data(file_path):
    """Excel 파일에서 뉴스 데이터를 로드합니다."""
    return pd.read_excel(file_path)

def load_dictionary(file_path):
    """JSON 파일에서 경제 용어 데이터를 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def extract_keywords(news_data, dictionary):
    """뉴스 데이터에서 키워드를 추출합니다."""
    keywords = set()  # 중복 방지를 위해 set 사용

    for _, row in news_data.iterrows():
        title = row['title']
        message = row['message']
        
        # 뉴스 제목과 메시지에서 각 경제 용어가 포함되어 있는지 확인
        for term in dictionary.keys():
            if term in title or term in message:
                keywords.add(term)

    return list(keywords)

In [ ]:
def save_keywords(keywords, file_path):
    """키워드를 JSON 파일로 저장합니다."""
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(keywords, f, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    # 데이터 경로 설정
    news_file_path = "news_results.xlsx"
    dict_file_path = "dictdb.json"
    keywords_file_path = "keywords.json"

    # 데이터 로드
    news_data = load_news_data(news_file_path)
    dictionary = load_dictionary(dict_file_path)

    # 키워드 추출
    keywords = extract_keywords(news_data, dictionary)

    # 결과 저장
    save_keywords(keywords, keywords_file_path)
    print(f"추출된 키워드를 '{keywords_file_path}'에 저장했습니다.")